In [ ]:
import time
import instagram_scraper as insta
import json
import pandas as pd
import config

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [ ]:
perfil_insta = ["cortefiel_official", 
                "scalperscompany", 
                "bluebananabrand", 
                "womensecretofficial", 
                "pedrodelhierro_official", 
                "fiftyoutlet_official", 
                "springfieldmw", 
                'goiko', 
                'sushiyakuza', 
                'goikobasics', 
                'zalando', 
                'zalando_man', 
                'superga_official', 
                'zara', 
                'pullandbear', 
                'massimodutti', 
                'oysho', 
                'silbon',
                'zarahome']
number_last_photos = 10000
list_media_likes = []
list_media_comments = []
list_timestamp = []
list_is_video = []
list_fullname = []

dbtype = config.database_new['dbtype']
user = config.database_new['user']
password = config.database_new['password']
ip = config.database_new['ip']
port = config.database_new['port']
name = config.database_new['name']

engine = create_engine(f'{dbtype}://{user}:{password}@{ip}:{port}/{name}')

def get_metadata():
    imgScraper = insta.InstagramScraper(usernames=[perfil_insta[x]], maximum=number_last_photos, profile_metadata=True, media_metadata=True, latest=False, media_types=['none'])
    imgScraper.scrape()
    print ("scraped " + str(number_last_photos) + " from " + perfil_insta[x])
    
def scrape_data():
    with open(perfil_insta[x] + '/' + perfil_insta[x] + '.json', 'r', encoding = 'utf8') as data_file:    
        data = json.load(data_file)
    data_access = data['GraphImages']
    for i in range(len(data_access)):
        list_media_likes.append(data_access[i]['edge_media_preview_like']['count'])
        list_media_comments.append(data_access[i]['edge_media_to_comment']['count'])
        list_timestamp.append(data_access[i]['taken_at_timestamp'])
        list_is_video.append(data_access[i]['is_video'])
        list_fullname.append(data_access[i]['username'])
    return list_media_likes, list_media_comments, list_timestamp, list_is_video, list_fullname

def load_data(): 
    df = pd.DataFrame(list_media_likes, columns = ['likes']) 
    df['comments'] = pd.Series(list_media_comments)
    df['date'] = pd.Series(list_timestamp)
    df['is_video'] = pd.Series(list_is_video)
    df['full_name'] = pd.Series(list_fullname)
    df['date'] = pd.to_datetime(df['date'],unit='s')
    df.to_sql(name='instagram_competitors_posts',con=engine, index=False, if_exists='replace', method='multi', chunksize=110)
    
def main():
    get_metadata()
    scrape_data()
    load_data()

for x in range(len(perfil_insta)):
    main()
    time.sleep(1)